In [ ]:
from flask import Flask, Response, jsonify
import cv2
import torch
from flask_cors import CORS
from datetime import datetime
import warnings  # 경고 메시지를 숨기기 위한 모듈

# 경고 메시지 숨기기
warnings.filterwarnings("ignore")  # 모든 경고 메시지 비활성화
torch.set_printoptions(profile="none")  # PyTorch의 내부 경고 숨김

# Flask 애플리케이션 설정
app = Flask(__name__)
CORS(app)

# Load YOLO model (replace 'yolov5s' with the model you want to use)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', verbose=False)  # verbose=False로 경고 최소화

# OpenCV video capture
video_capture = cv2.VideoCapture(0)

# List to store detection periods
detection_periods = []

# Variables to track detection state
is_person_detected = False
detection_start_time = None

@app.route('/')
def index():
    return """
        <h1>YOLO Flask Webcam</h1>
        <p>Click the button below to navigate:</p>
        <button onclick="window.location.href='/video_feed_page'">View Live Feed</button>
        <button onclick="window.location.href='/detections'">Go to detections Page</button>
        <button onclick="window.location.href='http://localhost/'">Go to Localhost</button>
    """

@app.route('/video_feed_page')
def video_feed_page():
    return """
        <h1>Live Webcam Feed</h1>
        <iframe src="/video_feed" width="640" height="480" style="border:none;"></iframe>
        <br>
        <button onclick="window.location.href='/detections'">Go to detections Page</button>
        <button onclick="window.location.href='http://localhost/'">Go to Localhost</button>
    """
@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/detections', methods=['GET'])
def get_detections():
    """Endpoint to get the list of detection periods."""
    return jsonify({"detection_periods": detection_periods})

def generate_frames():
    global is_person_detected, detection_start_time
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        # Convert the frame to RGB (YOLO expects RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Run YOLO detection
        results = model(rgb_frame)

        # Filter results for 'person' class only
        person_results = results.pandas().xyxy[0]
        person_results = person_results[person_results['name'] == 'person']

        # Check detection state
        if not person_results.empty:  # Person detected
            if not is_person_detected:  # New detection started
                detection_start_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                is_person_detected = True
                print(f"[DETECTION START] Person detected at: {detection_start_time}")
        else:  # No person detected
            if is_person_detected:  # Detection ended
                detection_end_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                detection_periods.append({"start": detection_start_time, "end": detection_end_time})
                is_person_detected = False
                print(f"[DETECTION END] Person detection ended at: {detection_end_time}")

        # Annotate frame with YOLO results (for person only)
        for _, row in person_results.iterrows():
            x1, y1, x2, y2, conf, cls, name = row
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Draw bounding box
            cv2.putText(frame, f"{name} {conf:.2f}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Encode frame as JPEG
        _, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
